In [1]:
import sys
from os import path

# dir = path.abspath()
# sys.path.append(dir.parent)

# from goodreads.models.doc2vec import Doc2VecModel

In [2]:
import pandas as pd
import numpy as np

In [3]:
import sqlalchemy as sa
import mysql.connector

In [4]:
dir = path.abspath('__file__')
sys.path.append(path.dirname(path.dirname(dir)))

In [5]:
from setup import setup
connector = setup()

Current time:  2023-06-09 12:20:07


In [6]:
descr_query = sa.text(
    "select * from processed_description;"
)
descr = pd.read_sql_query(descr_query, con=connector.connect())

In [7]:
from models.doc2vec import Doc2VecModel
model = Doc2VecModel(descr, 'goodreads_book_id', 'processed_descr')
model.fit()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [8]:
recommends = model.get_recommendations(3)

In [9]:
recommends

[12296, 114982, 121792, 17567]

In [10]:
books_query = sa.text(
    "select * from books;"
)
books = pd.read_sql_query(books_query, con=connector.connect())

In [11]:
indices, _ = model.get_docvecs()

In [20]:
indices[recommends]

goodreads_book_id
12296      135
114982    7532
121792    8819
17567     7700
dtype: int64

In [21]:
books.iloc[indices[recommends]]

,goodreads_book_id,isbn,isbn13,original_publication_year,original_title,title,language_code,average_rating,ratings_count,description
135,12296,142437263,9780142437260.0,1850.0,The Scarlet Letter,The Scarlet Letter,eng,3.37,509883,"Set in 17th-century Puritan Boston, Massachuse..."
7532,114982,394895894,9780394895900.0,1985.0,The Ruby in the Smoke,"The Ruby in the Smoke (Sally Lockhart, #1)",eng,3.75,16214,Sally is sixteen and uncommonly pretty. Her kn...
8819,121792,1417642122,9781417642120.0,1991.0,The Revenge of the Baby-Sat: A Calvin and Hobb...,The Revenge of the Baby-Sat,eng,4.71,11503,The praise and popularity of Calvin and Hobbes...
7700,17567,1576469115,9781576469120.0,1909.0,A Girl of the Limberlost,"A Girl of the Limberlost (Limberlost, #2)",eng,4.14,13071,"Set amid Indiana's vast Limberlost Swamp, this..."


In [24]:
chosen = books.loc[books.goodreads_book_id == '3']

In [28]:
chosen.title.values[0]

"Harry Potter and the Sorcerer's Stone (Harry Potter, #1)"

In [30]:
recommended_book = books[['title', 'average_rating', 'description']].iloc[indices[recommends]]

In [45]:
recommended_book = recommended_book.reset_index()

In [48]:
recommended_book.drop(columns=['index'], inplace=True)
recommended_book

,title,average_rating,description
0,The Scarlet Letter,3.37,"Set in 17th-century Puritan Boston, Massachuse..."
1,"The Ruby in the Smoke (Sally Lockhart, #1)",3.75,Sally is sixteen and uncommonly pretty. Her kn...
2,The Revenge of the Baby-Sat,4.71,The praise and popularity of Calvin and Hobbes...
3,"A Girl of the Limberlost (Limberlost, #2)",4.14,"Set amid Indiana's vast Limberlost Swamp, this..."
